Author: Lester Hedges<br>
Email:&nbsp;&nbsp; lester.hedges@bristol.ac.uk

# Equilibration

A node to perform equilibration of a molecular system.

In [ ]:
import BioSimSpace as BSS

In [ ]:
node = BSS.Gateway.Node("A node to perform equilibration. Saves the equlibrated molecular configuration and RMD to file.")
node.addAuthor(name="Lester Hedges", email="lester.hedges@bristol.ac.uk", affiliation="University of Bristol")
node.setLicense("GPLv3")

Set the input requirements:

In [ ]:
node.addInput("files", BSS.Gateway.FileSet(help="a set of molecular input files"))
node.addInput("runtime", BSS.Gateway.Float(help="the run time in nanoseconds", minimum=0, maximum=10, default=0.2))
node.addInput("temperature_start", BSS.Gateway.Float(help="the initial temperature in Kelvin", minimum=0, maximum=1000, default=0))
node.addInput("temperature_end", BSS.Gateway.Float(help="the final temperature in Kelvin", minimum=0, maximum=1000, default=300))
node.addInput("restrain_backbone", BSS.Gateway.Boolean(help="whether to restrain the backbone", default=False))
node.addInput("residue", BSS.Gateway.String(help="the name of the molecular residue"))

Note `runtime`, `temperature-start`, and `temperature-end` are currently of type `BSS.Gateway.Float`. In future these will be additional types, i.e. `BSS.Gateway.Time`, `BSS.Gateway.Temperature`, allowing better handling for units and conversions.

We now need to define the output of the node. In this case we will return a set of files representing the equilibrated molecular system and a single text file that stores the root mean squared displacement of the molecule containing the specified residue name.

In [ ]:
node.addOutput("equilibrated", BSS.Gateway.FileSet(help="the equilibrated molecular system"))
node.addOutput("rmsd", BSS.Gateway.File(help="the RMSD of the molecule"))

If needed, here are some input files again. These can then be re-uploaded using the GUI.

AMBER files: [ala.crd](../input/ala.crd), [ala.top](../input/ala.top)

Now show the GUI.

In [ ]:
node.showControls()

Generate the molecular system.

In [ ]:
system = BSS.readMolecules(node.getInput("files"))

Set up the equilibration protocol.

(Note that the keyword arguments happen to have the same name as the input requirements. This need not be the case.)

In [ ]:
protocol = BSS.Protocol.Equilibration(runtime=node.getInput("runtime"), temperature_start=node.getInput("temperature_start"), temperature_end=node.getInput("temperature_end"), restrain_backbone=node.getInput("restrain_backbone"))

Start the MD equilibration.

In [ ]:
process = BSS.MD.run(system, protocol)

Get the equilibrated molecular system and write to file in the same format as the input.

In [ ]:
node.setOutput("equilibrated", BSS.saveMolecules("equilibrated", process.getSystem(block=True), system.fileFormat()))

Write the RMSD of the chosen molecule to file and set the output requirement.

In [ ]:
rmsd = process.getTrajectory().RMSD(frame=0, molecule=system[BSS.MolWithResName(node.getInput("residue").upper())])

with open("rmsd.txt", "w") as file:
    for index, value in enumerate(rmsd):
        file.write("%d %5.4f\n" % (index, value))
        
node.setOutput("rmsd", "rmsd.txt")

Validate the node.

In [ ]:
node.validate()